In [2]:
import importlib
from pathlib import Path
import json

pth = Path("./")
libs = [
    "digitalhub_core_container.entities.registries",
    "digitalhub_data_dbt.entities.registries",
    "digitalhub_data_nefertem.entities.registries",
    "digitalhub_ml_mlrun.entities.registries",
]
for lib in libs:

    # Import registry
    module = importlib.import_module(lib)

    # Spec schemas
    spec_registry = getattr(module, "spec_registry")
    for kind, mapper in spec_registry.items():

        # Determine entity type
        typ = None
        l = kind.split("+")
        if len(l) == 1:
            typ = "function"
        else:
            if l[1] == "run":
                typ = "run"
            else:
                typ = "task"
        if typ is None:
            raise

        # Make folder
        (pth / typ).mkdir(parents=True, exist_ok=True)

        # Import data model
        spec_module = importlib.import_module(mapper["module"])
        spec_model = getattr(spec_module, mapper["spec_params"])

        # Get and write schema
        json_schema = spec_model.schema_json()
        name = f"schema_{typ}_{kind}.json"
        text = json.dumps(json.loads(json_schema), indent=4)
        (pth / typ / name).write_text(text)